In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("ch3-01")

LangSmith 추적을 시작합니다.
[프로젝트명]
ch3-01


In [4]:
from langchain_teddynote.messages import stream_response
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [5]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


In [6]:
#출력 파서를 사용하지 않는 경우

from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

**중요한 내용 추출:**

1. **발신자:** 김철수 (chulsoo.kim@bikecorporation.me)
2. **수신자:** 이은채 (eunchae@teddyinternational.me)
3. **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. **요청 사항:**
   - ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. **미팅 제안:**
   - 날짜: 다음 주 화요일 (1월 15일)
   - 시간: 오전 10시
   - 장소: 귀사 사무실

In [7]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 봰 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date : str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [8]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"title": "Person", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\uba54\uc77c\uc744 \ubd30 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\uba54\uc77c \uc81c\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ud14d\uc2a4\ud2b8", "type": "string"}, "date": {"title": "Date", "description": "\ub

In [9]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

prompt = prompt.partial(format=parser.get_format_instructions())

In [10]:
chain = prompt | llm

In [11]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)


```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안함.",
  "date": "다음 주 화요일(1월 15일) 오전 10시"
}
```

In [12]:
structured_output = parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary="바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안함." date='다음 주 화요일(1월 15일) 오전 10시'


In [13]:
chain = prompt | llm | parser

In [14]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 이은채 대리에게 ZENESIS 자전거 모델에 대한 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안함.', date='1월 15일 오전 10시')

In [15]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

In [16]:
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 이은채 대리에게 ZENESIS 자전거 모델에 대한 브로슈어 요청과 유통 협력 가능성을 논의하기 위한 미팅을 제안했습니다. 미팅은 다음 주 화요일(1월 15일) 오전 10시에 귀사 사무실에서 진행될 예정입니다.', date='1월 15일 오전 10시')